In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import cv2
import zipfile
import shutil
import os
import glob
import scipy.stats as stats

#matplot 한글폰트 설정
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:\Windows\Fonts\GULIM.TTC").get_name()
plt.rc("font", family = font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"]= False

In [2]:
os.getcwd()

'C:\\Users\\acorn\\Desktop'

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome()

In [5]:
# 크롤링시 필요한 라이브러리 불러오기
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)


# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
# 입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num + 1
    else:
        return num + 9 * (num - 1)


# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(
            start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + \
                search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls


##########뉴스크롤링 시작###################

# 검색어 입력
search = input("검색할 키워드를 입력해주세요:")

# 검색 시작할 페이지 입력
# ex)1 =1페이지,2=2페이지...
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):"))
print("\n크롤링할 시작 페이지: ", page, "페이지")
# 검색 종료할 페이지 입력
# ex)1 =1페이지,2=2페이지...
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):"))
print("\n크롤링할 종료 페이지: ", page2, "페이지")

# naver url 생성
search_urls = makeUrl(search, page, page2)

## selenium으로 navernews만 뽑아오기##

# 버전에 상관 없이 os에 설치된 크롬 브라우저 사용
driver = webdriver.Chrome()
driver.implicitly_wait(3)

# selenium으로 검색 페이지 불러오기 #

naver_urls = []

for i in search_urls:
    driver.get(i)
    time.sleep(1)  # 대기시간 변경 가능

    # 네이버 기사 눌러서 제목 및 본문 가져오기#
    # 네이버 기사가 있는 기사 css selector 모아오기
    a = driver.find_elements(By.CSS_SELECTOR, 'a.info')

    # 위에서 생성한 css selector list 하나씩 클릭하여 본문 url얻기
    for i in a:
        i.click()

        # 현재탭에 접근
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(1)  # 대기시간 변경 가능

        # 네이버 뉴스 url만 가져오기

        url = driver.current_url
        print(url)

        if "news.naver.com" in url:
            naver_urls.append(url)

        else:
            pass
        # 현재 탭 닫기
        driver.close()
        # 다시처음 탭으로 돌아가기(매우 중요!!!)
        driver.switch_to.window(driver.window_handles[0])

print(naver_urls)

###naver 기사 본문 및 제목 가져오기###

# ConnectionError방지
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

titles = []
contents = []
for i in naver_urls:
    original_html = requests.get(i, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    # print(html)

    # 뉴스 제목 가져오기
    title = html.select(
        "div#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=title)
    titles.append(title)

    # # 뉴스 본문 가져오기

    # content = html.select("div#dic_area")

    # # 기사 텍스트만 가져오기
    # # list합치기
    # content = ''.join(str(content))

    # # html태그제거 및 텍스트 다듬기
    # content = re.sub(pattern=pattern1, repl='', string=content)
    # pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    # content = content.replace(pattern2, '')

    # contents.append(content)

print(titles)
# print(contents)

# 데이터프레임으로 정리(titles,url,contents)

# news_df = pd.DataFrame(
#     {'title': titles, 'link': naver_urls, 'content': contents})
news_df = pd.DataFrame(
    {'title': titles, 'link': naver_urls})

# news_df.to_csv('NaverNews_%s.csv' % search, index=False, encoding='utf-8-sig')


검색할 키워드를 입력해주세요:농업

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):101

크롤링할 시작 페이지:  101 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):200

크롤링할 종료 페이지:  200 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=1001', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=1011', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=1021', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=1031', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=1041', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=1051', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=1061', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=1071', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=1081', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=1091', 'https://search.nave

https://www.yna.co.kr/
https://n.news.naver.com/mnews/article/001/0014018391?sid=101
https://www.sedaily.com/
https://n.news.naver.com/mnews/article/011/0004205987?sid=101
http://www.4th.kr/
http://www.jemin.com/
http://www.foodnews.co.kr/
https://newsis.com/
https://n.news.naver.com/mnews/article/003/0011928929?sid=102
https://www.thefirstmedia.net/
https://news.imaeil.com/
https://n.news.naver.com/mnews/article/088/0000821739?sid=100
https://www.yna.co.kr/
https://n.news.naver.com/mnews/article/001/0014029357?sid=102
http://www.insightkorea.co.kr/
https://news.imaeil.com/
https://n.news.naver.com/mnews/article/088/0000821739?sid=100
http://www.thebigdata.co.kr/
https://www.yna.co.kr/
https://n.news.naver.com/mnews/article/001/0014029357?sid=102
http://www.insightkorea.co.kr/
https://www.straightnews.co.kr/
http://www.shinailbo.co.kr/
http://www.domin.co.kr/
http://www.domin.co.kr/bannerpop/popup.html?win_popup_idxno=1943
https://www.upinews.kr/
http://www.bzeronews.com/
https://www.k

http://www.ccnnews.co.kr/
http://www.ccnnews.co.kr/
http://www.ccnnews.co.kr/bannerpop/popup.html?win_popup_idxno=3095
http://www.ccnnews.co.kr/bannerpop/popup.html?win_popup_idxno=3095
https://www.ccdailynews.com/
http://www.siminilbo.co.kr/
https://www.news1.kr/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://n.news.naver.com/mnews/article/421/0006879204?sid=102
http://www.ceoscoredaily.com/
http://www.chungnamilbo.co.kr/
http://www.domin.co.kr/
http://www.domin.co.kr/bannerpop/popup.html?win_popup_idxno=1943
https://www.gukjenews.com/
https://newsis.com/
https://n.news.naver.com/mnews/article/003/0011933696?sid=102
https://www.thekpm.com/
https://www.hankyung.com/
https://n.news.naver.com/mnews/article/015/0004859810?sid=102
https://www.fnnews.com/
https://n.news.naver.com/mnews/article/014/0005031786?sid=102
https://www.nongmin.com/
https://n.news.naver.com/mnews/article/662/0000023239?sid=102
https://newsis.com/
https://n.news.naver.com/mnews/article/

http://www.issuenbiz.com/
https://newsis.com/
https://n.news.naver.com/mnews/article/003/0011929283?sid=102
https://www.foodneconomy.com/
http://www.hidomin.com/
https://www.cctoday.co.kr/
https://www.dailian.co.kr/
https://n.news.naver.com/mnews/article/119/0002724532?sid=101
http://www.kbsm.net/
https://www.nongmin.com/
https://n.news.naver.com/mnews/article/662/0000023049?sid=102
https://www.cctoday.co.kr/bannerpop/popup.html?win_popup_idxno=3532
http://www.enewstoday.co.kr/
https://www.news1.kr/
https://n.news.naver.com/mnews/article/421/0006876743?sid=102
http://www.chukkyung.co.kr/
https://www.kwnews.co.kr/
https://n.news.naver.com/mnews/article/087/0000979698?sid=102
https://www.newspim.com/
http://www.issuenbiz.com/
https://www.ccdn.co.kr/
https://www.ccdn.co.kr/bannerpop/popup.html?win_popup_idxno=2252
http://www.dynews.co.kr/
https://www.nongmin.com/
https://n.news.naver.com/mnews/article/662/0000022838?sid=101
http://www.newsworker.co.kr/
http://www.shinailbo.co.kr/
http://w

http://www.bzeronews.com/
https://www.nocutnews.co.kr/
https://n.news.naver.com/mnews/article/079/0003782824?sid=102
https://www.nocutnews.co.kr/pop/pop230626.htm
https://www.tf.co.kr/
https://n.news.naver.com/mnews/article/629/0000224182?sid=102
http://www.sjbnews.com/
http://www.agrinet.co.kr/
https://www.asiae.co.kr/
https://n.news.naver.com/mnews/article/277/0005275407?sid=101
http://www.shinailbo.co.kr/
http://www.kjdaily.com/
https://www.newspim.com/
http://biz.heraldcorp.com/
https://n.news.naver.com/mnews/article/016/0002159207?sid=101
https://www.newsfreezone.co.kr/
https://www.idomin.com/
http://www.cfnews.kr/coding/main.aspx
https://www.asiatoday.co.kr/
https://www.edaily.co.kr/
https://www.edaily.co.kr/popup/viewpopup.html?v=3
https://n.news.naver.com/mnews/article/018/0005512722?sid=100
https://www.dnews.co.kr/
http://www.ccnnews.co.kr/
https://newsis.com/
http://www.ccnnews.co.kr/bannerpop/popup.html?win_popup_idxno=3095
https://n.news.naver.com/mnews/article/003/00119105

http://www.dtnews24.com/
http://www.jeonmin.co.kr/
http://www.jeonmin.co.kr/
https://www.breaknews.com/
https://www.yna.co.kr/
https://n.news.naver.com/mnews/article/001/0014002415?sid=101
http://www.gnnews.co.kr/
https://www.sportsseoul.com/
https://n.news.naver.com/mnews/article/468/0000953768?sid=102
https://www.betanews.net/
https://www.hankookilbo.com/
https://n.news.naver.com/mnews/article/469/0000745238?sid=102
http://www.efnews.co.kr/
https://chmbc.co.kr/
http://www.aflnews.co.kr/
http://www.aflnews.co.kr/bannerpop/popup.html?win_popup_idxno=441
http://www.aflnews.co.kr/
http://www.jbnews.com/
http://www.aflnews.co.kr/bannerpop/popup.html?win_popup_idxno=441
https://www.kwnews.co.kr/
https://n.news.naver.com/mnews/article/087/0000979550?sid=004
http://www.jeonmin.co.kr/
https://www.jeonmae.co.kr/
https://www.dailian.co.kr/
https://n.news.naver.com/mnews/article/119/0002724203?sid=004
https://www.khan.co.kr/
https://n.news.naver.com/mnews/article/032/0003231953?sid=103
https://w

http://www.seoulwire.com/
https://www.seoul.co.kr/
https://n.news.naver.com/mnews/article/081/0003371892?sid=102
https://www.ekn.kr/web/
https://www.mbn.co.kr/
https://n.news.naver.com/mnews/article/057/0001750341?sid=102
http://www.jbnews.com/
https://www.kwnews.co.kr/
https://n.news.naver.com/mnews/article/087/0000978816?sid=102
https://www.news1.kr/
https://n.news.naver.com/mnews/article/421/0006884189?sid=102
http://www.joongdo.co.kr/web/
http://www.joongdo.co.kr/web/popup1.php
https://www.nocutnews.co.kr/
https://www.nocutnews.co.kr/pop/pop230626.htm
https://n.news.naver.com/mnews/article/079/0003783568?sid=102
https://www.newscj.com/
https://www.breaknews.com/
http://www.siminilbo.co.kr/
http://www.hidomin.com/
https://www.fnnews.com/
https://n.news.naver.com/mnews/article/014/0005026079?sid=100
https://www.tf.co.kr/
https://n.news.naver.com/mnews/article/629/0000223728?sid=102
https://www.metroseoul.co.kr/
https://www.metroseoul.co.kr/
http://www.jejuilbo.net/
https://www.incheo

["[농협 삼척시지부, 원덕서 '농업인 행복버스' 운영]", '[정황근 농식품부 장관, 서아프리카 방문해 韓 농업기술 전파]', '[국가 예산 대비 농업예산 5% 확보하라!]', '[농식품부, 배수개선 사업지 32만㏊로 확대… 농업생산기반 정비계획 발표]', "['스마트 강소농' 육성 본격화…익산시, 농업인 대상 15차례 교육]", '[농식품부, 배수개선 사업지 32만㏊로 확대… 농업생산기반 정비계획 발표]', "['스마트 강소농' 육성 본격화…익산시, 농업인 대상 15차례 교육]", '[강원특별자치도농업기술원, 고랭지감자 농가에 역병 방제 당부]', '[강원특별자치도농업기술원, 고랭지감자 농가에 역병 방제 당부]', '[[포토뉴스]강원농협, 결혼이민여성 일대일 맞춤형 농업교육 워크숍]', '[[인터뷰] “보훈·농업, 국가안보·식량안보와 직결”]', '[영양군, 2024년 농업기술특화시범사업 참여 농업인 공모]', '[전남도, 여름철 농업재해 예방 대책 수립]', '[[민선8기 1년] 김태흠 충남지사 "임기 내 청년농업인 3천명 육성"]', '[“농촌 물관리 강화”…배수개선 32만㏊로 확대·농업용수 관리 ICT 활용]', '[농업생산기반 정비 10개년 계획 발표]', '[[창원소식]경남농협, 이민여성 1대 1 맞춤형 농업교육 등]', '[정황근 농림장관, 아프리카 방문...한국 농업기술 전파]', '[남양주풍양보건소 치매안심센터, 농업기술센터와 함께 치유농업 프로그램 운영]', '[광주농업기술센터, 찾아가는 반려식물병원 운영]', '[정황근 장관, 한국 농업기술 전파 통한 아프리카 식량안보 협력 방안 논의]', '["농업소득 두배로"…경북도, \'농업 대전환\' 나섰다]', '["농업소득 두배로"…경북도, \'농업 대전환\' 나섰다]', '[울주 두북농협, 농업인행복센터 문 열어]', '[농식품부, 농업생산 기반 10개년 정비계획 수립 발표]', '[제주도 후계농업경영인대회 ‘성료’]', '[농식품부, 농업생산 기반 10개년 정비계획 수립 발표]', '[제주도

In [6]:
news_df.to_csv("C:/Users/acorn/Desktop/크롤링데이터/naver_news_smartfarm101-200.csv",
               encoding='utf-8-sig', index=False)

In [7]:
news_df

,title,link
0,"[농협 삼척시지부, 원덕서 '농업인 행복버스' 운영]",https://n.news.naver.com/mnews/article/001/001...
1,"[정황근 농식품부 장관, 서아프리카 방문해 韓 농업기술 전파]",https://n.news.naver.com/mnews/article/011/000...
2,[국가 예산 대비 농업예산 5% 확보하라!],https://n.news.naver.com/mnews/article/003/001...
3,"[농식품부, 배수개선 사업지 32만㏊로 확대… 농업생산기반 정비계획 발표]",https://n.news.naver.com/mnews/article/088/000...
4,"['스마트 강소농' 육성 본격화…익산시, 농업인 대상 15차례 교육]",https://n.news.naver.com/mnews/article/001/001...
...,...,...
265,[[녹색] 통합관제시스템 연계 노지 스마트 농업기술 시연회],https://n.news.naver.com/mnews/article/052/000...
266,[강원특별자치도 농업용수 수질조사 결과 모두 ‘적합’],https://n.news.naver.com/mnews/article/087/000...
267,[주말농장족 한숨 돌렸네…농업 활동 농막 야간취침 허용],https://n.news.naver.com/mnews/article/003/001...
268,"[우리농업지키기운동 대구본부, 농산물꾸러미 나눔]",https://n.news.naver.com/mnews/article/662/000...


In [8]:
# 크롤링시 필요한 라이브러리 불러오기
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)


# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
# 입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num + 1
    else:
        return num + 9 * (num - 1)


# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(
            start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + \
                search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls


##########뉴스크롤링 시작###################

# 검색어 입력
search = input("검색할 키워드를 입력해주세요:")

# 검색 시작할 페이지 입력
# ex)1 =1페이지,2=2페이지...
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):"))
print("\n크롤링할 시작 페이지: ", page, "페이지")
# 검색 종료할 페이지 입력
# ex)1 =1페이지,2=2페이지...
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):"))
print("\n크롤링할 종료 페이지: ", page2, "페이지")

# naver url 생성
search_urls = makeUrl(search, page, page2)

## selenium으로 navernews만 뽑아오기##

# 버전에 상관 없이 os에 설치된 크롬 브라우저 사용
driver = webdriver.Chrome()
driver.implicitly_wait(3)

# selenium으로 검색 페이지 불러오기 #

naver_urls = []

for i in search_urls:
    driver.get(i)
    time.sleep(1)  # 대기시간 변경 가능

    # 네이버 기사 눌러서 제목 및 본문 가져오기#
    # 네이버 기사가 있는 기사 css selector 모아오기
    a = driver.find_elements(By.CSS_SELECTOR, 'a.info')

    # 위에서 생성한 css selector list 하나씩 클릭하여 본문 url얻기
    for i in a:
        i.click()

        # 현재탭에 접근
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(1)  # 대기시간 변경 가능

        # 네이버 뉴스 url만 가져오기

        url = driver.current_url
        print(url)

        if "news.naver.com" in url:
            naver_urls.append(url)

        else:
            pass
        # 현재 탭 닫기
        driver.close()
        # 다시처음 탭으로 돌아가기(매우 중요!!!)
        driver.switch_to.window(driver.window_handles[0])

print(naver_urls)

###naver 기사 본문 및 제목 가져오기###

# ConnectionError방지
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

titles = []
contents = []
for i in naver_urls:
    original_html = requests.get(i, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    # print(html)

    # 뉴스 제목 가져오기
    title = html.select(
        "div#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=title)
    titles.append(title)

    # # 뉴스 본문 가져오기

    # content = html.select("div#dic_area")

    # # 기사 텍스트만 가져오기
    # # list합치기
    # content = ''.join(str(content))

    # # html태그제거 및 텍스트 다듬기
    # content = re.sub(pattern=pattern1, repl='', string=content)
    # pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    # content = content.replace(pattern2, '')

    # contents.append(content)

print(titles)
# print(contents)

# 데이터프레임으로 정리(titles,url,contents)

# news_df = pd.DataFrame(
#     {'title': titles, 'link': naver_urls, 'content': contents})
news_df = pd.DataFrame(
    {'title': titles, 'link': naver_urls})

# news_df.to_csv('NaverNews_%s.csv' % search, index=False, encoding='utf-8-sig')


검색할 키워드를 입력해주세요:농업

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):201

크롤링할 시작 페이지:  201 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):300

크롤링할 종료 페이지:  300 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=2001', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=2011', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=2021', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=2031', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=2041', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=2051', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=2061', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=2071', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=2081', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=2091', 'https://search.nave

http://www.newsworker.co.kr/
https://www.nongmin.com/
https://n.news.naver.com/mnews/article/662/0000023012?sid=102
https://www.asiae.co.kr/
https://n.news.naver.com/mnews/article/277/0005269405?sid=100
https://www.ccdn.co.kr/
https://www.ccdn.co.kr/bannerpop/popup.html?win_popup_idxno=2252
http://www.worktoday.co.kr/
http://www.cfnews.kr/coding/main.aspx
http://kr.people.com.cn/
https://www.news1.kr/
https://n.news.naver.com/mnews/article/421/0006887529?sid=103
https://www.yna.co.kr/
https://n.news.naver.com/mnews/article/001/0014003827?sid=105
http://www.bzeronews.com/
https://www.cnbnews.com/
https://www.incheonilbo.com/
https://www.nspna.com/
https://www.ohmynews.com/
https://n.news.naver.com/mnews/article/047/0002396180?sid=100
https://daily.hankooki.com/
https://www.news1.kr/
https://n.news.naver.com/mnews/article/421/0006866245?sid=102
https://www.gukjenews.com/
http://www.bzeronews.com/
http://www.weeklytoday.com/
http://www.headlinejeju.co.kr/
https://www.yna.co.kr/
https://n.

https://www.seoul.co.kr/
https://n.news.naver.com/mnews/article/081/0003368842?sid=101
https://www.dkilbo.com/
https://www.wowtv.co.kr/
https://n.news.naver.com/mnews/article/215/0001107490?sid=101
https://www.ccdailynews.com/
http://www.shinailbo.co.kr/
http://www.chungnamilbo.co.kr/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
http://www.ccnnews.co.kr/
https://www.ccdailynews.com/
http://www.ccnnews.co.kr/bannerpop/popup.html?win_popup_idxno=3095
http://www.joongdo.co.kr/web/
http://www.joongdo.co.kr/web/popup1.php
http://www.joongdo.co.kr/web/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
http://www.joongdo.co.kr/web/popup1.php
http://www.econonews.co.kr/
http://www.fntoday.co.kr/
http://www.joongboo.com/
https://www.etnews.com/
https://n.news.naver.com/mnews/article/030/0003109848?sid=102
https://www.ekn.kr/web/
https://www.asiatoday.co.kr/
http://www.jejuilbo.net/
http://www.jjn.co.kr/
https://newsis.com/
https://n.news.naver.com/mn

https://www.fnnews.com/
https://n.news.naver.com/mnews/article/014/0005026496?sid=102
http://www.aflnews.co.kr/
http://www.aflnews.co.kr/bannerpop/popup.html?win_popup_idxno=441
http://www.joongboo.com/
http://www.enewstoday.co.kr/
http://www.daejonilbo.com/
https://n.news.naver.com/mnews/article/656/0000054392?sid=102
https://www.gukjenews.com/
https://www.newsfreezone.co.kr/
http://www.newsmaker.or.kr/
https://www.discoverynews.kr/
https://www.nocutnews.co.kr/
https://n.news.naver.com/mnews/article/079/0003780847?sid=102
http://www.weeklytoday.com/
https://www.gukjenews.com/
http://www.ksmnews.co.kr/default/index.php
https://www.nocutnews.co.kr/pop/pop230626.htm
https://www.segye.com/
https://n.news.naver.com/mnews/article/022/0003822935?sid=110
http://www.socialvalue.kr/
https://www.metroseoul.co.kr/
http://www.agrinet.co.kr/
http://www.siminilbo.co.kr/
https://www.dkilbo.com/
https://www.dkilbo.com/
https://www.sedaily.com/
https://n.news.naver.com/mnews/article/011/0004200549?sid=

https://n.news.naver.com/mnews/article/025/0003286890?sid=102
https://www.yna.co.kr/
https://n.news.naver.com/mnews/article/001/0013985784?sid=100
https://www.newsfreezone.co.kr/
https://www.aitimes.kr/
https://www.nongmin.com/
https://n.news.naver.com/mnews/article/662/0000022183?sid=104
https://www.yna.co.kr/
https://n.news.naver.com/mnews/article/001/0013985784?sid=100
https://www.newsfreezone.co.kr/
https://www.aitimes.kr/
https://www.nongmin.com/
https://n.news.naver.com/mnews/article/662/0000022183?sid=104
http://www.chungnamilbo.co.kr/
https://www.ccdn.co.kr/
https://www.ccdn.co.kr/bannerpop/popup.html?win_popup_idxno=2252
https://www.edaily.co.kr/
https://www.edaily.co.kr/popup/viewpopup.html?v=9
https://n.news.naver.com/mnews/article/018/0005508435?sid=103
https://www.metroseoul.co.kr/
http://www.chungnamilbo.co.kr/
https://www.ccdailynews.com/
http://www.shinailbo.co.kr/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
http://www.aflnews.co.kr/
http://www

https://newsis.com/
https://n.news.naver.com/mnews/article/003/0011909361?sid=102
https://www.ekn.kr/web/
http://www.ksmnews.co.kr/default/index.php
http://www.ccnnews.co.kr/
http://www.ccnnews.co.kr/bannerpop/popup.html?win_popup_idxno=3095
https://www.news1.kr/
https://n.news.naver.com/mnews/article/421/0006870359?sid=101
http://www.shinailbo.co.kr/
https://www.yna.co.kr/
https://n.news.naver.com/mnews/article/001/0013986113?sid=101
http://www.cfnews.kr/coding/main.aspx
https://www.yna.co.kr/
https://n.news.naver.com/mnews/article/001/0013986052?sid=101
https://www.asiatoday.co.kr/
https://www.asiatoday.co.kr/
https://www.dkilbo.com/
http://www.enewstoday.co.kr/
https://www.metroseoul.co.kr/
https://www.ccdn.co.kr/
https://www.ccdn.co.kr/bannerpop/popup.html?win_popup_idxno=2252
https://chmbc.co.kr/
http://www.ebuzz.co.kr/
http://www.amnews.co.kr/
https://daily.hankooki.com/
http://www.sjbnews.com/
http://www.newsworker.co.kr/
https://www.ccdn.co.kr/
https://www.ccdn.co.kr/bannerpop/

https://www.news1.kr/
https://n.news.naver.com/mnews/article/421/0006853191?sid=102
https://www.ajunews.com/
https://www.kwnews.co.kr/
https://n.news.naver.com/mnews/article/087/0000977589?sid=102
http://www.issuenbiz.com/
http://www.dynews.co.kr/
http://www.ccnnews.co.kr/
https://daily.hankooki.com/
http://www.ccnnews.co.kr/bannerpop/popup.html?win_popup_idxno=3095
http://www.aflnews.co.kr/
http://www.aflnews.co.kr/bannerpop/popup.html?win_popup_idxno=441
https://www.joongang.co.kr/
https://n.news.naver.com/mnews/article/025/0003289263?sid=110
https://news.imaeil.com/
https://n.news.naver.com/mnews/article/088/0000818291?sid=100
https://www.ebn.co.kr/
https://www.kwnews.co.kr/
https://n.news.naver.com/mnews/article/087/0000978562?sid=102
https://www.dnews.co.kr/
http://www.weeklytoday.com/
https://www.jeonmae.co.kr/
http://www.jbnews.com/
http://www.jbnews.com/
http://www.domin.co.kr/
http://www.enewstoday.co.kr/
https://www.sedaily.com/
http://www.domin.co.kr/bannerpop/popup.html?win

['[서울 강동농협, 대학생농업체험단 운영 재개, “농촌 관심 있는 대학생 참여해주세요~”]', "['이게 바로 자율주행 이앙기'... 尹, 청년 농업인과 모내기]", '[북한 농업과학연구소 "과학 농사로 재해성 이상 기후 극복"]', '[KIST 천연물연구소, 몽골 전통 약용식물 및 농업기술 개발 협력]', '[여주시농업기술센터, 농촌지도사업 육성 거점센터 선정]', '[김영환 지사, 서울대와 AI 스마트농업 협력 제안]', '["우크라이나 농업 생산량 회복에 20년 이상 걸릴 것"]', '[농식품부, 여성농업인 생활수기 공모…다문화여성도 참여 가능]', '[귀농 농업인 창업 및 주택구입 지원 관심]', '[바이오·농업 선순환 생태계 구축…강원도, 그린바이오 선도기업 육성]', '[“치유농업시설 평가 기준 간소화…유효기간 5년으로 연장을”]', '[[열혈 기초단체장에게 듣는다] “청년농에 정착비·교육훈련비 지원…친환경농업 적극 육성”]', '[경기도농업기술원, 쌀맥주에 이어 딸기맥주도 개발]', "[전북도 전국 첫 '농업기계산업 육성·지원 조례안' 상임위 통과]", "[전북도 전국 첫 '농업기계산업 육성·지원 조례안' 상임위 통과]", '[군산시 농업용 면세유 보조금 추가 지급]', '[광주농업기술센터, 품목별 선도농업인 육성]', "[안성시, 오이농가 대상 '스마트 농업분야 기술보급' 평가회 개최]", '[동해시, 농업인수당 70만원 동해페이로 지급]', '[(사)우리농업지키기운동본부, 농협 인천옹진군지부와 함께 농촌일손 봉사]', "[[데일리 북한] 오늘도 전원회의 결정 '관철' 강조…농업 성과 독려]", '[밀양시, 스마트6차 농업수도 조성…부서 핵심사업 교류]', '[[정선소식] 우리농업지키기운동본부, 선풍기 100대 지원]', '[경남농협·39사단, 농업·농촌 발전을 위해 맞손]', "[화성시 농업기술센터, 美환경자원학회 토양 분야 국제숙련도 평가에서 '최우수']", '[강원도, 임대형 스마트팜 청년농업인 8팀 24명 선발]', "[신안군 '청년농업

In [9]:
news_df.to_csv("C:/Users/acorn/Desktop/크롤링데이터/naver_news_smartfarm201-300.csv",
               encoding='utf-8-sig', index=False)

In [10]:
# 크롤링시 필요한 라이브러리 불러오기
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)


# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
# 입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num + 1
    else:
        return num + 9 * (num - 1)


# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(
            start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + \
                search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls


##########뉴스크롤링 시작###################

# 검색어 입력
search = input("검색할 키워드를 입력해주세요:")

# 검색 시작할 페이지 입력
# ex)1 =1페이지,2=2페이지...
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):"))
print("\n크롤링할 시작 페이지: ", page, "페이지")
# 검색 종료할 페이지 입력
# ex)1 =1페이지,2=2페이지...
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):"))
print("\n크롤링할 종료 페이지: ", page2, "페이지")

# naver url 생성
search_urls = makeUrl(search, page, page2)

## selenium으로 navernews만 뽑아오기##

# 버전에 상관 없이 os에 설치된 크롬 브라우저 사용
driver = webdriver.Chrome()
driver.implicitly_wait(3)

# selenium으로 검색 페이지 불러오기 #

naver_urls = []

for i in search_urls:
    driver.get(i)
    time.sleep(1)  # 대기시간 변경 가능

    # 네이버 기사 눌러서 제목 및 본문 가져오기#
    # 네이버 기사가 있는 기사 css selector 모아오기
    a = driver.find_elements(By.CSS_SELECTOR, 'a.info')

    # 위에서 생성한 css selector list 하나씩 클릭하여 본문 url얻기
    for i in a:
        i.click()

        # 현재탭에 접근
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(1)  # 대기시간 변경 가능

        # 네이버 뉴스 url만 가져오기

        url = driver.current_url
        print(url)

        if "news.naver.com" in url:
            naver_urls.append(url)

        else:
            pass
        # 현재 탭 닫기
        driver.close()
        # 다시처음 탭으로 돌아가기(매우 중요!!!)
        driver.switch_to.window(driver.window_handles[0])

print(naver_urls)

###naver 기사 본문 및 제목 가져오기###

# ConnectionError방지
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

titles = []
contents = []
for i in naver_urls:
    original_html = requests.get(i, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    # print(html)

    # 뉴스 제목 가져오기
    title = html.select(
        "div#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=title)
    titles.append(title)

    # # 뉴스 본문 가져오기

    # content = html.select("div#dic_area")

    # # 기사 텍스트만 가져오기
    # # list합치기
    # content = ''.join(str(content))

    # # html태그제거 및 텍스트 다듬기
    # content = re.sub(pattern=pattern1, repl='', string=content)
    # pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    # content = content.replace(pattern2, '')

    # contents.append(content)

print(titles)
# print(contents)

# 데이터프레임으로 정리(titles,url,contents)

# news_df = pd.DataFrame(
#     {'title': titles, 'link': naver_urls, 'content': contents})
news_df = pd.DataFrame(
    {'title': titles, 'link': naver_urls})

# news_df.to_csv('NaverNews_%s.csv' % search, index=False, encoding='utf-8-sig')


검색할 키워드를 입력해주세요:농업

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):301

크롤링할 시작 페이지:  301 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):400

크롤링할 종료 페이지:  400 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=3001', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=3011', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=3021', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=3031', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=3041', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=3051', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=3061', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=3071', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=3081', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=3091', 'https://search.nave

http://www.gnmaeil.com/
http://www.domin.co.kr/
http://www.domin.co.kr/bannerpop/popup.html?win_popup_idxno=1943
https://www.hankookilbo.com/
https://n.news.naver.com/mnews/article/469/0000743360?sid=101
https://www.obs.co.kr/
http://www.siminsori.com/
http://www.seoulfn.com/
http://www.enewstoday.co.kr/
https://www.inthenews.co.kr/index.html
https://www.newspim.com/
http://www.joongdo.co.kr/web/
http://www.joseilbo.com/index.php
http://www.joongdo.co.kr/web/popup1.php
https://n.news.naver.com/mnews/article/123/0002308433?sid=101
https://news.kbs.co.kr/common/main.html
https://n.news.naver.com/mnews/article/056/0011505525?sid=101
http://www.enewstoday.co.kr/
http://www.kjdaily.com/
https://www.metroseoul.co.kr/
http://www.shinailbo.co.kr/
https://www.psnews.co.kr/
http://www.jemin.com/
https://www.idaegu.co.kr/
https://www.newsworks.co.kr/
http://www.domin.co.kr/
http://www.econonews.co.kr/
http://www.domin.co.kr/bannerpop/popup.html?win_popup_idxno=1943
http://www.kyongbuk.co.kr/
http

http://www.chungnamilbo.co.kr/
https://www.moneys.co.kr/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://n.news.naver.com/mnews/article/417/0000927592?sid=102
https://www.dailian.co.kr/
https://n.news.naver.com/mnews/article/119/0002719222?sid=101
https://www.gukjenews.com/
http://www.kbsm.net/
http://biz.heraldcorp.com/
https://n.news.naver.com/mnews/article/016/0002156160?sid=102
http://www.dynews.co.kr/
http://www.joongdo.co.kr/web/
http://www.joongdo.co.kr/web/popup1.php
https://news.kbs.co.kr/common/main.html
https://n.news.naver.com/mnews/article/056/0011503355?sid=101
https://www.nspna.com/
https://tfmedia.co.kr/
https://www.etnews.com/
https://n.news.naver.com/mnews/article/030/0003106036?sid=101
http://www.ksmnews.co.kr/default/index.php
http://www.gndomin.com/
https://www.fnnews.com/
https://n.news.naver.com/mnews/article/014/0005026481?sid=101
https://newsis.com/
https://n.news.naver.com/mnews/article/003/0011898227?sid=102
https://www.asiatoday

http://www.bzeronews.com/
http://www.bbsi.co.kr/HOME/
https://www.mk.co.kr/
https://n.news.naver.com/mnews/article/009/0005144612?sid=101
http://biz.heraldcorp.com/
https://n.news.naver.com/mnews/article/016/0002155835?sid=100
http://www.hortitimes.com/
https://www.kmaeil.com/
https://www.ngetnews.com/
https://www.ngetnews.com/
https://www.hankookilbo.com/
https://n.news.naver.com/mnews/article/469/0000742984?sid=102
http://www.weeklytoday.com/
http://www.joongdo.co.kr/web/
http://www.joongdo.co.kr/web/popup1.php
https://www.lawissue.co.kr/
http://www.foodnews.co.kr/
https://www.segye.com/
https://n.news.naver.com/mnews/article/022/0003822819?sid=101
http://www.kookje.co.kr/
https://n.news.naver.com/mnews/article/658/0000042798?sid=102
https://www.breaknews.com/
http://www.domin.co.kr/
http://www.domin.co.kr/bannerpop/popup.html?win_popup_idxno=1943
http://www.joongdo.co.kr/web/
http://www.joongdo.co.kr/web/popup1.php
https://www.lcnews.co.kr/
http://www.aflnews.co.kr/
http://www.aflne

https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
http://www.kihoilbo.co.kr/
https://www.yna.co.kr/
https://n.news.naver.com/mnews/article/001/0013990147?sid=102
https://www.kwnews.co.kr/
https://n.news.naver.com/mnews/article/087/0000977576?sid=102
https://www.yna.co.kr/
https://n.news.naver.com/mnews/article/001/0013990145?sid=102
https://www.kwnews.co.kr/
https://n.news.naver.com/mnews/article/087/0000977576?sid=102
http://www.ngonews.kr/
https://www.yna.co.kr/
https://n.news.naver.com/mnews/article/001/0013990827?sid=102
http://www.gndomin.com/
https://www.pressian.com/
https://n.news.naver.com/mnews/article/002/0002289428?sid=102
http://www.aflnews.co.kr/
http://www.aflnews.co.kr/bannerpop/popup.html?win_popup_idxno=441
https://www.getnews.co.kr/
https://www.cnbnews.com/
http://www.kihoilbo.co.kr/
http://www.kihoilbo.co.kr/
https://www.nspna.com/
http://www.jjan.kr/
https://kjmbc.co.kr/
https://www.newsfreezone.co.kr/
https://www.sportsseoul.com/
https://n.news

https://www.nbntv.co.kr/
http://www.aflnews.co.kr/
http://www.shinailbo.co.kr/
http://www.jejuilbo.net/
https://www.kwnews.co.kr/
http://www.aflnews.co.kr/bannerpop/popup.html?win_popup_idxno=441
https://n.news.naver.com/mnews/article/087/0000976125?sid=102
https://www.kwnews.co.kr/
https://n.news.naver.com/mnews/article/087/0000975250?sid=102
http://news.lghellovision.net/
http://www.aflnews.co.kr/
http://www.bzeronews.com/
http://www.aflnews.co.kr/bannerpop/popup.html?win_popup_idxno=441
http://www.4th.kr/
https://newsis.com/
https://n.news.naver.com/mnews/article/003/0011905140?sid=102
https://www.kukinews.com/
http://www.jnilbo.com/
https://ysmbc.co.kr/
https://www.inews365.com/
https://www.ajunews.com/
http://www.kyeongin.com/main/index.php
http://www.joongdo.co.kr/web/
http://www.joongdo.co.kr/web/popup1.php
https://www.gukjenews.com/
https://www.kwnews.co.kr/
https://n.news.naver.com/mnews/article/087/0000976918?sid=102
https://www.mbceg.co.kr/
http://www.knnews.co.kr/autoi/inde

https://www.inews365.com/
http://www.jnilbo.com/
https://www.newspim.com/
https://www.jeonmae.co.kr/
https://www.metroseoul.co.kr/
https://www.newsfreezone.co.kr/
http://www.aflnews.co.kr/
http://www.aflnews.co.kr/bannerpop/popup.html?win_popup_idxno=441
http://www.wonyesanup.co.kr/
http://www.aflnews.co.kr/
http://www.aflnews.co.kr/bannerpop/popup.html?win_popup_idxno=441
https://www.asiatoday.co.kr/
https://www.inews365.com/
https://www.nongmin.com/
https://n.news.naver.com/mnews/article/662/0000021694?sid=102
https://www.dkilbo.com/
http://www.domin.co.kr/
http://www.domin.co.kr/bannerpop/popup.html?win_popup_idxno=1943
['https://n.news.naver.com/mnews/article/469/0000743360?sid=101', 'https://n.news.naver.com/mnews/article/123/0002308433?sid=101', 'https://n.news.naver.com/mnews/article/056/0011505525?sid=101', 'https://n.news.naver.com/mnews/article/003/0011902899?sid=102', 'https://n.news.naver.com/mnews/article/003/0011913495?sid=102', 'https://n.news.naver.com/mnews/article/005

['[美 농기계 회사 이어 네덜란드 스마트팜...SK네트웍스 농업 스타트업에 공들이다]', "[김수흥 의원, 농업 발전 위한 '농업식품기본법' 등 개정안 발의]", '[[지방소멸 시대] 청년 인구 유치 주력…농업·관광도시 조성]', '[[청주소식] 농업용 취입보 3곳 개·보수 등]', "['농가 부채 1억원 육박' 제주 농업지표 악화…대책 주문]", '[농협 하나로마트, ‘고객이 기운나게! 농업인이 살맛나게!’할인 행사]', '[김제시, 2023년도 농업경영체 전수조사 실시]', "[남영준 강원4-H연합회장, '함께하는 청년농업인상' 수상]", '[경남농업기술원, 양파 신품종 ‘아크네’ 현장평가회]', '[“30년후 100세 시대 겨냥 ‘생명과학입국’ 선언 시급” [2023 한국농업 미래혁신포럼]]', '[두산밥캣, 美 농업 신기술 개발 회사에 지분투자…자율주행 기술 개발 속도]', '[[청바지] 농업에 드론 활용 ‘단비 드론’]', '[[전남 청년 마을로 프로젝트 수혜기업] &lt;16&gt;소규모 농가와 동행하는 해남 농업회사법인 ‘다올]', '[강원특별자치도농업기술원, 고추냉이 우량종묘 채종 완료]', '[“농업용 저수지 등 물관리 시설 보강” 촉구]', '[두산밥캣, 美 농업 원격·무인 기술 기업에 지분 투자]', '[우리농업지키기운동본부 전남지역본부, 사랑의 우리 농산물꾸러미 나눔 행사]', "[고성 시범농장 만화방초, '치유농업 프로그램' 10일부터 가동]", '[[동정] 원정숙 횡성군농업인학습단체 한마음 대회장]', '[창고서 충전 중이던 농업용 드론 배터리 폭발 화재]', '[G20 농업장관회의 16~17일 개최…지속가능한 농식품시스템 등 논의]', '[농진청, 국내외 ‘유전자교정기술’ 전문가와 농업생명 국제 화상토론회]', '[지속 가능 농업환경 조성하는 전남도...유기농 생태마을 4개소 신규 지정]', '[두산밥캣, 美 농업 신기술 회사에 지분투자]', '[서울시-NH농협손해보험, 도시농업 활성화 맞손]', '[NH진도군농촌현장봉사단 농업인행

In [11]:
news_df.to_csv("C:/Users/acorn/Desktop/크롤링데이터/naver_news_smartfarm301-400.csv",
               encoding='utf-8-sig', index=False)

In [12]:
news_df

,title,link
0,[美 농기계 회사 이어 네덜란드 스마트팜...SK네트웍스 농업 스타트업에 공들이다],https://n.news.naver.com/mnews/article/469/000...
1,"[김수흥 의원, 농업 발전 위한 '농업식품기본법' 등 개정안 발의]",https://n.news.naver.com/mnews/article/123/000...
2,[[지방소멸 시대] 청년 인구 유치 주력…농업·관광도시 조성],https://n.news.naver.com/mnews/article/056/001...
3,[[청주소식] 농업용 취입보 3곳 개·보수 등],https://n.news.naver.com/mnews/article/003/001...
4,['농가 부채 1억원 육박' 제주 농업지표 악화…대책 주문],https://n.news.naver.com/mnews/article/003/001...
...,...,...
179,"[경남농협, ‘함께하는 올해의 청년농업인像’ 2명 선정]",https://n.news.naver.com/mnews/article/277/000...
180,[[2023 K-농산어촌 한마당①] K농업·K농촌 발전 가능성을 맛 보다],https://n.news.naver.com/mnews/article/016/000...
181,"[농기계조합, 필리핀과 농업기계 협력 파트너십 MOU 체결…필리핀 전용공단 설립 탄...",https://n.news.naver.com/mnews/article/662/000...
182,"[[지방소멸 시대] “‘백종원 효과’ 성공적…예산군, 농업 중심지 육성”]",https://n.news.naver.com/mnews/article/056/001...


In [13]:
# 크롤링시 필요한 라이브러리 불러오기
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)


# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
# 입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num + 1
    else:
        return num + 9 * (num - 1)


# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(
            start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + \
                search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls


##########뉴스크롤링 시작###################

# 검색어 입력
search = input("검색할 키워드를 입력해주세요:")

# 검색 시작할 페이지 입력
# ex)1 =1페이지,2=2페이지...
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):"))
print("\n크롤링할 시작 페이지: ", page, "페이지")
# 검색 종료할 페이지 입력
# ex)1 =1페이지,2=2페이지...
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):"))
print("\n크롤링할 종료 페이지: ", page2, "페이지")

# naver url 생성
search_urls = makeUrl(search, page, page2)

## selenium으로 navernews만 뽑아오기##

# 버전에 상관 없이 os에 설치된 크롬 브라우저 사용
driver = webdriver.Chrome()
driver.implicitly_wait(3)

# selenium으로 검색 페이지 불러오기 #

naver_urls = []

for i in search_urls:
    driver.get(i)
    time.sleep(1)  # 대기시간 변경 가능

    # 네이버 기사 눌러서 제목 및 본문 가져오기#
    # 네이버 기사가 있는 기사 css selector 모아오기
    a = driver.find_elements(By.CSS_SELECTOR, 'a.info')

    # 위에서 생성한 css selector list 하나씩 클릭하여 본문 url얻기
    for i in a:
        i.click()

        # 현재탭에 접근
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(1)  # 대기시간 변경 가능

        # 네이버 뉴스 url만 가져오기

        url = driver.current_url
        print(url)

        if "news.naver.com" in url:
            naver_urls.append(url)

        else:
            pass
        # 현재 탭 닫기
        driver.close()
        # 다시처음 탭으로 돌아가기(매우 중요!!!)
        driver.switch_to.window(driver.window_handles[0])

print(naver_urls)

###naver 기사 본문 및 제목 가져오기###

# ConnectionError방지
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

titles = []
contents = []
for i in naver_urls:
    original_html = requests.get(i, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    # print(html)

    # 뉴스 제목 가져오기
    title = html.select(
        "div#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=title)
    titles.append(title)

    # # 뉴스 본문 가져오기

    # content = html.select("div#dic_area")

    # # 기사 텍스트만 가져오기
    # # list합치기
    # content = ''.join(str(content))

    # # html태그제거 및 텍스트 다듬기
    # content = re.sub(pattern=pattern1, repl='', string=content)
    # pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    # content = content.replace(pattern2, '')

    # contents.append(content)

print(titles)
# print(contents)

# 데이터프레임으로 정리(titles,url,contents)

# news_df = pd.DataFrame(
#     {'title': titles, 'link': naver_urls, 'content': contents})
news_df = pd.DataFrame(
    {'title': titles, 'link': naver_urls})

# news_df.to_csv('NaverNews_%s.csv' % search, index=False, encoding='utf-8-sig')


검색할 키워드를 입력해주세요:농업

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):401

크롤링할 시작 페이지:  401 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):500

크롤링할 종료 페이지:  500 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=4001', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=4011', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=4021', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=4031', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=4041', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=4051', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=4061', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=4071', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=4081', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=농업&start=4091', 'https://search.nave

https://www.ccdailynews.com/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
http://www.ikpnews.net/
https://www.nspna.com/
https://www.nspna.com/
https://news.kbs.co.kr/common/main.html
https://n.news.naver.com/mnews/article/056/0011498121?sid=101
https://www.gukjenews.com/
https://www.newsfreezone.co.kr/
https://www.thepublic.kr/
https://www.kmaeil.com/
https://www.segye.com/
https://n.news.naver.com/mnews/article/022/0003819972?sid=110
https://www.ccdailynews.com/
http://www.ikpnews.net/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://www.nspna.com/
https://www.nspna.com/
https://news.kbs.co.kr/common/main.html
https://n.news.naver.com/mnews/article/056/0011498121?sid=101
https://www.gukjenews.com/
https://www.newsfreezone.co.kr/
https://www.thepublic.kr/
https://www.kmaeil.com/
https://www.segye.com/
https://n.news.naver.com/mnews/article/022/0003819972?sid=110
https://www.ccdailynews.com/
http://www.ikpnews.net/
https://www.nspna

https://www.nspna.com/
https://www.nspna.com/
https://news.kbs.co.kr/common/main.html
https://n.news.naver.com/mnews/article/056/0011498121?sid=101
https://www.gukjenews.com/
https://www.newsfreezone.co.kr/
https://www.thepublic.kr/
http://www.jeonmin.co.kr/
https://www.ccdailynews.com/
https://www.ccdailynews.com/
http://www.ikpnews.net/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://www.nspna.com/
https://www.nspna.com/
https://news.kbs.co.kr/common/main.html
https://n.news.naver.com/mnews/article/056/0011498121?sid=101
https://www.gukjenews.com/
https://www.newsfreezone.co.kr/
https://www.thepublic.kr/
http://www.jeonmin.co.kr/
https://www.ccdailynews.com/
https://www.ccdailynews.com/
http://www.ikpnews.net/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://www.nspna.com/
https://www.nsp

https://www.nspna.com/
https://news.kbs.co.kr/common/main.html
https://n.news.naver.com/mnews/article/056/0011498121?sid=101
https://www.gukjenews.com/
https://www.newsfreezone.co.kr/
https://www.thepublic.kr/
http://www.jeonmin.co.kr/
https://www.ccdailynews.com/
https://www.ccdailynews.com/
http://www.ikpnews.net/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://www.nspna.com/
https://www.nspna.com/
https://news.kbs.co.kr/common/main.html
https://n.news.naver.com/mnews/article/056/0011498121?sid=101
https://www.gukjenews.com/
https://www.newsfreezone.co.kr/
https://www.thepublic.kr/
http://www.jeonmin.co.kr/
https://www.ccdailynews.com/
https://www.ccdailynews.com/
http://www.ikpnews.net/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://www.nspna.com/
https://www.nspna.com/
https://news.kb

https://www.newsfreezone.co.kr/
http://www.ccnnews.co.kr/
http://www.ccnnews.co.kr/bannerpop/popup.html?win_popup_idxno=3095
http://www.jeonmin.co.kr/
https://www.ccdailynews.com/
https://www.ccdailynews.com/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
http://www.ikpnews.net/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://www.nspna.com/
https://www.nspna.com/
https://news.kbs.co.kr/common/main.html
https://n.news.naver.com/mnews/article/056/0011498121?sid=101
https://www.gukjenews.com/
https://www.newsfreezone.co.kr/
http://www.ccnnews.co.kr/
http://www.ccnnews.co.kr/bannerpop/popup.html?win_popup_idxno=3095
http://www.jeonmin.co.kr/
https://www.ccdailynews.com/
https://www.ccdailynews.com/
http://www.ikpnews.net/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://www.nspna.com/
https://www.nspna.com/
https://news.kbs.co.kr/common/mai

http://www.ccnnews.co.kr/bannerpop/popup.html?win_popup_idxno=3095
https://www.ccdailynews.com/
http://www.jeonmin.co.kr/
https://www.ccdailynews.com/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://www.nspna.com/
https://www.nspna.com/
http://www.ikpnews.net/
https://news.kbs.co.kr/common/main.html
https://n.news.naver.com/mnews/article/056/0011498121?sid=101
https://www.gukjenews.com/
https://www.newsfreezone.co.kr/
http://www.ccnnews.co.kr/
http://www.ccnnews.co.kr/bannerpop/popup.html?win_popup_idxno=3095
https://www.ccdailynews.com/
http://www.jeonmin.co.kr/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://www.ccdailynews.com/
https://www.nspna.com/
https://www.nspna.com/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
http://www.ikpnews.net/
https://news.kbs.co.kr/common/main.html
https://n.news.naver.com/mnews/article/056/00114981

https://www.ccdailynews.com/
http://www.jeonmin.co.kr/
http://www.ccnnews.co.kr/bannerpop/popup.html?win_popup_idxno=3095
https://www.ccdailynews.com/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://www.nspna.com/
https://www.nspna.com/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
http://www.ikpnews.net/
https://news.kbs.co.kr/common/main.html
https://n.news.naver.com/mnews/article/056/0011498121?sid=101
https://www.gukjenews.com/
http://www.idaegu.com/
http://www.ccnnews.co.kr/
https://www.ccdailynews.com/
http://www.ccnnews.co.kr/bannerpop/popup.html?win_popup_idxno=3095
http://www.jeonmin.co.kr/
https://www.ccdailynews.com/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
https://www.nspna.com/
https://www.nspna.com/
https://www.ccdailynews.com/bannerpop/popup.html?win_popup_idxno=1486
http://www.ikpnews.net/
https://news.kbs.co.kr/common/main.html
https://n.news.naver.com/mnews/article/056/0011498121?sid=10

In [14]:
news_df.to_csv("C:/Users/acorn/Desktop/크롤링데이터/naver_news_smartfarm401-500.csv",
               encoding='utf-8-sig', index=False)

In [15]:
news_df

,title,link
0,[지역농업 ‘선순환’ 농촌융복합산업…내실화 ‘과제’],https://n.news.naver.com/mnews/article/056/001...
1,"[[기고] 생명환경농업, 국가 전략산업으로 육성해야]",https://n.news.naver.com/mnews/article/022/000...
2,[지역농업 ‘선순환’ 농촌융복합산업…내실화 ‘과제’],https://n.news.naver.com/mnews/article/056/001...
3,"[[기고] 생명환경농업, 국가 전략산업으로 육성해야]",https://n.news.naver.com/mnews/article/022/000...
4,[지역농업 ‘선순환’ 농촌융복합산업…내실화 ‘과제’],https://n.news.naver.com/mnews/article/056/001...
...,...,...
89,[지역농업 ‘선순환’ 농촌융복합산업…내실화 ‘과제’],https://n.news.naver.com/mnews/article/056/001...
90,[지역농업 ‘선순환’ 농촌융복합산업…내실화 ‘과제’],https://n.news.naver.com/mnews/article/056/001...
91,[지역농업 ‘선순환’ 농촌융복합산업…내실화 ‘과제’],https://n.news.naver.com/mnews/article/056/001...
92,[지역농업 ‘선순환’ 농촌융복합산업…내실화 ‘과제’],https://n.news.naver.com/mnews/article/056/001...
